In [82]:
import numpy as np
import pandas as pd

## Load COLVAR

In [83]:
def plumed_to_pandas(filename):
    headers = pd.read_csv(filename,sep=' ',skipinitialspace=True, nrows=0).columns[2:]  
    df = pd.read_csv(filename,sep=' ',skipinitialspace=True, header=None,skiprows=1,names=headers,comment='#') 
    return df

colvar = plumed_to_pandas('data/chignolin-v3/COLVAR')
colvar = colvar.rename(columns={'deep.node-4': 'tica1','deep.node-3': 'tica2','deep.node-2': 'tica3', 'deep.node-1': 'tica4','deep.node-0': 'tica5'})
colvar = colvar[::10]
colvar

,time,rmsd_ca,end,hbonds,tica5,tica4,tica3,tica2,tica1
0,0.0,0.067477,0.500523,3.515683,0.156591,0.269741,-0.503362,-0.769360,0.860714
10,10.0,0.040256,0.516848,3.607478,-0.163258,0.031106,0.738618,-0.639663,0.860408
20,20.0,0.062648,0.460648,3.625588,-0.114638,0.244315,0.888992,-0.485358,0.901081
30,30.0,0.067292,0.490304,3.519522,-0.082365,-0.005303,-0.405775,-0.523814,0.901542
40,40.0,0.065588,0.487023,3.634205,0.348991,-0.240763,-0.353688,-0.586350,0.871337
...,...,...,...,...,...,...,...,...,...
524700,524700.0,0.651776,1.920228,0.413543,0.028602,0.230118,0.027031,-0.283929,-0.689612
524710,524710.0,0.543898,1.960420,0.512699,-0.170959,0.161725,-0.014535,-0.523711,-0.522852
524720,524720.0,0.428343,1.464966,0.604115,-0.156648,0.097244,0.012294,-0.381107,-0.555181
524730,524730.0,0.452596,1.448579,0.586075,-0.098890,0.154325,0.106024,-0.387451,-0.569581


## FES with KDE

draft of n-dimensional method

In [85]:
def compute_fes(colvar, collective_vars, bounds, num=100, bw_method = 0.05, kbt=2.5, logweights=None ):
    
    ndims = len(collective_vars)
    sigma = [bw_method for _ in range(ndims)]

    _1d_grid = [np.linspace(vmin, vmax, num) for (vmin, vmax) in bounds]
    meshgrids = np.meshgrid(*_1d_grid)
    coords = np.array([np.ravel(coord) for coord in meshgrids]).T

    #compute probability with KDE
    max_prob=0
    prob = np.zeros(coords.shape[0]) #np.zeros([num for _ in range(ndims)])

    for i,sample in enumerate(coords):
        arg2 = 0
        for j in range(ndims):
            dx_j = sample[j] - colvar[collective_vars[j]]
            arg2 += np.power(dx_j/sigma[j],2)
            if logweights:
                prob[i] = np.sum(np.exp(logweights)*np.exp(-0.5*arg2))
            else:
                prob[i] = np.sum(np.exp(-0.5*arg2))
            if prob[i] > max_prob:
                max_prob = prob[i]

    prob = np.reshape(prob,(num,)*ndims)

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)

    return fes

collective_vars = ['tica1','tica2']
bounds = [(-1.,1.)]*len(collective_vars)

fes = compute_fes(colvar,collective_vars, bounds, num=100, kbt=2.8)


### 1D and 2D case

(not adapted)

In [ ]:
# 1D CASE: support derivatives, free energy difference, and periodic CVs

def compute_fes_1d(cv_x,logweights,kbt,sigma,extent,grid_bin=100,compute_derivatives=False,periodic=False,compute_deltaF=False,deltaF_max=0.):
    #define grid
    gx_min,gx_max = extent
    grid_x=np.linspace(gx_min,gx_max,grid_bin)
    
    #compute probability
    max_prob=0
    prob=np.zeros(grid_bin)
    deriv=[]
    if compute_derivatives:
        deriv=np.zeros(grid_bin)
    basinA,basinB=0,0
    period=0
    if periodic:
        period=gx_max-gx_min
    if periodic and compute_derivatives:
        raise AttributeError('derivatives not implemented for periodic variables')
        
    for i in range(grid_bin):
        dx,arg2=0,0
        if periodic:
            dx=np.absolute(grid_x[i]-cv_x)
            arg2=(np.minimum(dx,period-dx)/sigma)**2
        else:
            dx=grid_x[i]-cv_x
            arg2=(dx/sigma)**2
        prob[i]=np.sum(np.exp(logweights)*np.exp(-0.5*arg2))
        if compute_derivatives:
            deriv[i]=np.sum(-dx/(sigma**2)*np.exp(logweights)*np.exp(-0.5*arg2))
        if prob[i]>max_prob:
            max_prob=prob[i]
        if compute_deltaF:
            if grid_x[i]<deltaF_max:
                basinA+=prob[i]
            else:
                basinB+=prob[i]

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)
    
    if compute_derivatives and compute_deltaF:
        deriv=-kbt*deriv/prob
        deltaF=kbt*np.log(basinA/basinB)
        return grid_x,fes,deriv,deltaF
    elif compute_deltaF:
        deltaF=kbt*np.log(basinA/basinB)
        return grid_x,fes,deltaF
    else:
        return grid_x,fes

# 2D case

def compute_fes_2d(cv_x,cv_y,logweights,kbt,sigma,extent,grid_bin=100):
    #define grid
    gx_min,gx_max,gy_min,gy_max = extent
    xx=np.linspace(gx_min,gx_max,grid_bin)
    yy=np.linspace(gy_min,gy_max,grid_bin)
    grid_x,grid_y=np.meshgrid(xx,yy)

    #retrieve_sigma
    if isinstance(sigma, list):
        sigma_x,sigma_y=sigma
    else:
        sigma_x=sigma
        sigma_y=sigma
        
    #compute probability
    max_prob=0
    prob=np.zeros((grid_bin,grid_bin))
    for i in range(grid_bin):
        for j in range(grid_bin):
            dx=np.absolute(grid_x[i,j]-cv_x)
            dy=np.absolute(grid_y[i,j]-cv_y)
            arg2=(dx/sigma_x)**2+(dy/sigma_y)**2
            prob[i,j]=np.sum(np.exp(logweights)*np.exp(-0.5*arg2)) 
            if prob[i,j]>max_prob:
                max_prob=prob[i,j]

    #convert to fes
    fes=-kbt*np.log(prob/max_prob)
    
    return fes